# Data Wrangling

This notebook presents different data wrangling techniques used commonly

In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

from IPython.display import display

pd.options.mode.chained_assignment = None

## Utilities

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """
    
    # sentinels
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

## Generate a Sample Dataset

In [3]:
df = generate_sample_data(row_count=1000)

### Describe the Dataset

In [4]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int64
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int64
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 60


Sample Indices with missing data:: [0, 1, 2, 4, 6]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int64
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int64
Price                 969 non-null float64


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:98: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:101: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,17,NaN,n
1,1001,NaN,5554,323,27,5683.14,n
2,1002,NaN,5255,932,29,NaN,n
3,1003,2016-01-21,5911,812,9,5174.73,n
4,1004,NaN,5159,496,17,2741.24,n


### Rename Columns

In [5]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [6]:
cleanup_column_names(df)

In [7]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


### Sort Rows on defined attributes

In [8]:
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
648,-1,2016-10-01,5929,541,23,5486.28,d
736,-1,2016-10-01,5928,476,18,5151.47,d
893,-1,2016-01-14,5420,137,28,4168.12,c
447,-1,2016-01-19,5469,751,1,3849.35,a
99,-1,2016-08-01,5743,159,8,3385.55,b


### Rearrange Columns in a Dataframe

In [9]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,17,NaN
1,1001,NaN,5554,n,323,27,5683.14
2,1002,NaN,5255,n,932,29,NaN
3,1003,2016-01-21,5911,n,812,9,5174.73
4,1004,NaN,5159,n,496,17,2741.24


### Filtering Columns

Using Column Index

In [10]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[  0 323 932 812 496 205 830 199 178 884]


Using Column Name

In [11]:
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])

[17 27 29  9 17  7 38 25 31 18]


Using Column Datatype

In [12]:
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[    nan 5683.14     nan 5174.73 2741.24 4825.19  699.55 1678.85 1055.88
 3496.21]


### Filtering Rows

Select specific rows

In [13]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-01-25,5930,943,18,35.85,n
501,1501,2016-01-02,5554,323,22,1936.00,c
20,1020,2016-01-31,5395,132,41,211.02,n


Exclude Specific Row indices

In [14]:
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5554,323,27,5683.14,n
2,1002,NaN,5255,932,29,NaN,n
3,1003,2016-01-21,5911,812,9,5174.73,n
4,1004,NaN,5159,496,17,2741.24,n
5,1005,2016-01-27,5745,205,7,4825.19,n


Conditional Filtering

In [15]:
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5554,323,27,5683.14,n
2,1002,NaN,5255,932,29,NaN,n
6,1006,NaN,5825,830,38,699.55,n
8,1008,NaN,5087,178,31,1055.88,n
11,1011,2016-09-01,5363,792,33,1737.31,n


Offset from top of the dataframe

In [16]:
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-09-01,5847,1065,37,2715.61,b
101,1101,2016-01-14,5554,323,3,257.02,a
102,1102,2016-07-01,5255,932,13,579.26,b
103,1103,NaN,5911,812,19,884.73,b
104,1104,2016-01-29,5159,496,18,4629.23,c


Offset from bottom of the dataframe

In [17]:
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-08-01,5612,535,14,4998.85,a
991,1991,2016-01-13,5283,524,29,1119.20,a
992,1992,2016-09-01,5257,540,29,2168.74,d
993,1993,2016-05-02,5420,137,31,1368.62,b
994,1994,2016-07-01,5924,842,7,2012.13,b


### TypeCasting/Data Type Conversion

In [18]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int64
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int64
price                        float64
user_type                     object
dtype: object


### Apply/Map Usage

Map : Create a derived attribute using map

In [19]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-01-19,5586,618,22,3938.70,a,new
996,1996,2016-01-22,5539,212,21,1829.56,d,loyal_existing
997,1997,2016-01-27,5735,260,32,535.73,b,new
998,1998,2016-01-13,5159,890,2,2295.37,c,existing
999,1999,2016-01-31,5743,159,11,1678.35,a,new


Apply: Using apply to get attribute ranges

In [20]:
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

serial_no             2000.00
user_id               6087.00
product_id            1095.00
quantity_purchased      40.00
price                 7661.98
dtype: float64

Applymap: Extract week from date

In [21]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [22]:
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,17,NaN,n,error,0
1,1001,NaT,5554,323,27,5683.14,n,error,0
2,1002,NaT,5255,932,29,NaN,n,error,0
3,1003,2016-01-21,5911,812,9,5174.73,n,error,3
4,1004,NaT,5159,496,17,2741.24,n,error,0


### Missing Values

Drop Rows with missing dates

In [23]:
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
3,1003,2016-01-21,5911,812,9,5174.73,n,error,3
5,1005,2016-01-27,5745,205,7,4825.19,n,error,4
9,1009,2016-01-19,5954,884,18,3496.21,n,error,3
10,1010,2016-01-25,5930,943,18,35.85,n,error,4
11,1011,2016-09-01,5363,792,33,1737.31,n,error,35


Fill Missing Price values with mean price

In [24]:
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

Fill Missing user_type values with value from previous row (forward fill) 

In [25]:
df_dropped['user_type'].fillna(method='ffill',inplace=True)

Fill Missing user_type values with value from next row (backward fill)

In [26]:
df_dropped['user_type'].fillna(method='bfill',inplace=True)

### Duplicates

Drop Duplicate serial_no rows

In [27]:
# sample duplicates
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
99,-1,2016-08-01,5743,159,8,3385.55,b,new,31
195,-1,2016-01-26,5586,618,13,1119.54,d,loyal_existing,4
218,-1,2016-10-02,5762,576,18,1588.02,d,loyal_existing,39
236,-1,2016-01-29,5928,476,33,519.56,c,existing,4
242,-1,2016-07-02,5585,765,11,1658.62,d,loyal_existing,26


Shape of df=(969, 9)


In [28]:
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [29]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
3,1003,2016-01-21,5911,812,9,5174.73,n,error,3
5,1005,2016-01-27,5745,205,7,4825.19,n,error,4
9,1009,2016-01-19,5954,884,18,3496.21,n,error,3
10,1010,2016-01-25,5930,943,18,35.85,n,error,4
11,1011,2016-09-01,5363,792,33,1737.31,n,error,35


Shape of df=(940, 9)


Remove rows which have less than 3 attributes with non-missing data

In [30]:
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,17,NaN,n,error,0
1,1001,NaT,5554,323,27,5683.14,n,error,0
2,1002,NaT,5255,932,29,NaN,n,error,0
3,1003,2016-01-21,5911,812,9,5174.73,n,error,3
4,1004,NaT,5159,496,17,2741.24,n,error,0


Shape of df=(1000, 9)


### Encode Categoricals

One Hot Encoding using get_dummies()

In [31]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,17,NaN,error,0,0,0,0,0,1
1,1001,NaT,5554,323,27,5683.14,error,0,0,0,0,0,1
2,1002,NaT,5255,932,29,NaN,error,0,0,0,0,0,1
3,1003,2016-01-21,5911,812,9,5174.73,error,3,0,0,0,0,1
4,1004,NaT,5159,496,17,2741.24,error,0,0,0,0,0,1


Label Mapping

In [32]:
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,1995,2016-01-19,5586,618,22,3938.70,a,new,3,0.0
996,1996,2016-01-22,5539,212,21,1829.56,d,loyal_existing,3,3.0
997,1997,2016-01-27,5735,260,32,535.73,b,new,4,1.0
998,1998,2016-01-13,5159,890,2,2295.37,c,existing,2,2.0
999,1999,2016-01-31,5743,159,11,1678.35,a,new,4,0.0


### Random Sampling data from DataFrame

In [33]:
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
102,1102,2016-07-01,5255,932,13,579.26,b,new,26,1.0
435,1435,2016-01-20,5431,776,40,1714.68,c,existing,3,2.0
860,1860,2016-11-01,5901,228,34,2732.73,a,new,44,0.0
270,1270,2016-01-19,5251,978,16,71.51,b,new,3,1.0
106,1106,2016-01-30,5825,830,18,5101.04,c,existing,4,2.0


### Normalizing Numeric Values

Normalize price values using  **Min-Max Scaler**

In [34]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [35]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
16,1016,2016-01-23,5556,916,18,0.261772,a,new,3,0.0
21,1021,2016-01-27,5553,674,35,0.319355,a,new,4,0.0
25,-1,2016-01-21,5024,726,10,0.289372,b,new,3,1.0
28,1028,2016-06-02,5847,832,25,0.510547,c,existing,22,2.0
32,1032,2016-12-01,5475,680,36,0.963800,b,new,48,1.0


Normalize quantity purchased values using  **Robust Scaler**

In [36]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [37]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
16,1016,2016-01-23,5556,916,-0.15,2014.75,a,new,3,0.0
21,1021,2016-01-27,5553,674,0.70,2455.95,a,new,4,0.0
25,-1,2016-01-21,5024,726,-0.55,2226.22,b,new,3,1.0
28,1028,2016-06-02,5847,832,0.20,3920.86,c,existing,22,2.0
32,1032,2016-12-01,5475,680,0.75,7393.68,b,new,48,1.0


### Data Summarization

Condition based aggregation

In [38]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2363.954686192467


Condtion based counts

In [39]:
print(df['purchase_week'].value_counts())

4     172
3     163
2      96
35     61
44     58
5      55
26     47
53     45
31     45
39     44
22     41
9      38
13     36
0      31
17     24
48     24
18     20
Name: purchase_week, dtype: int64


### Group By

Group By certain attributes

In [40]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error               689
existing           4357
loyal_existing     5633
new               10421
Name: quantity_purchased, dtype: int64


Group By with different aggregate functions

In [41]:
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,689,22.966667,30
existing,4357,19.894977,219
loyal_existing,5633,19.975177,282
new,10421,22.219616,469


Group by specific aggregate functions for each attribute

In [42]:
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2253.755714                  41
existing       c          2265.795628                  41
loyal_existing d          2367.868860                  41
new            a          2363.954686                  41
               b          2315.601116                  41

Group by with multiple agg for each attribute

In [43]:
display(df.groupby(['user_class','user_type']).agg({'price':{
                                                                'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            63105.16  2253.755714    1831.066027   30.0   
existing       c           487146.06  2265.795628    1570.048192  219.0   
loyal_existing d           644060.33  2367.868860    1561.379157  282.0   
new            a           564985.17  2363.954686    1647.208092  244.0   
               b           497854.24  2315.601116    1520.603322  225.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        689  
existing       c                       4357  
loyal_existing d                       5633  
new            a                       5362  
               b                       5059

### Pivot Tables

In [44]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,3224.676667,1019.416000,1949.403333,3745.146667,NaN
2016-01-02,1623.212857,2338.280000,1520.405000,2173.094545,NaN
2016-01-13,2082.502000,1003.850000,2360.626667,1875.572857,NaN
2016-01-14,3017.952000,1974.005000,1970.871667,2288.596667,649.190
2016-01-15,1563.687143,2212.843333,2424.177500,3604.202000,NaN
2016-01-16,1442.952500,2384.233750,2944.070000,1606.576667,NaN
2016-01-17,969.536667,2028.758333,873.580000,2947.130000,NaN
2016-01-18,2307.803333,1895.987143,2881.228571,1623.364000,942.460
2016-01-19,4024.035000,1854.923333,2935.028000,2456.080000,3496.210


### Stack a Dataframe

In [45]:
print(df.stack())

0    serial_no                            1000
     user_id                              -101
     product_id                              0
     quantity_purchased                     17
     user_type                               n
     user_class                          error
     purchase_week                           0
1    serial_no                            1001
     user_id                              5554
     product_id                            323
     quantity_purchased                     27
     price                             5683.14
     user_type                               n
     user_class                          error
     purchase_week                           0
2    serial_no                            1002
     user_id                              5255
     product_id                            932
     quantity_purchased                     29
     user_type                               n
     user_class                          error
     purchase